In [1]:
...from time import sleep
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
import requests 
import csv

In [2]:
driver_exe_path = "/home/geethanjana/Datascience/SC599/chromedriver_linux64/chromedriver"
initial_url = "https://www.mayoclinic.org/diseases-conditions/"

In [3]:
chrome_options = Options()
chrome_options.add_argument("--headless")

driver = webdriver.Chrome(driver_exe_path, options=chrome_options)
driver.get(initial_url);

In [4]:
url_list = set()

def capture_links():
    category_links = driver.find_elements_by_tag_name('a');
        
    for category_link in category_links:
        category_href = category_link.get_attribute('href')
        if category_href.startswith("https://www.mayoclinic.org/diseases-conditions/index?letter=A"):
            sub_driver = webdriver.Chrome(driver_exe_path, options=chrome_options)
            sub_driver.get(category_href)
            sleep(2)
            disease_links = sub_driver.find_elements_by_xpath("//a[contains(@href,'/symptoms-causes/')]")
            for disease_link in disease_links:
                disease_href = disease_link.get_attribute('href')
                url_list.add(disease_href)
                
            sub_driver.quit()
      

capture_links()
driver.quit()

In [22]:

xurl = 'https://www.mayoclinic.org/diseases-conditions/latex-allergy/symptoms-causes/syc-20374287'
response = requests.get(xurl)
full_html = bs(response.content, "html.parser")
Disease = full_html.find("h1").text
result = u""

for tag in full_html.find("h2",text="Symptoms").next_siblings:
    if tag.name == "h2":
        break
    else:
        result += str(tag)
        
cropped_html = bs(result)

In [66]:
with open("data/dataset_2.csv", mode="w", newline="", encoding='utf-8') as out_file:
    dataset = csv.writer(out_file, delimiter=",", quotechar='"', quoting = csv.QUOTE_MINIMAL)
    dataset.writerow(["Disease", "Symptom_1", "Symptom_2", "Symptom_3", "Symptom_4", "Symptom_5", "Symptom_6", "Symptom_7", "Symptom_8",
                      "Symptom_9", "Symptom_10", "Symptom_11", "Symptom_12", "Symptom_13", "Symptom_14", "Symptom_15", "Symptom_16", "Symptom_17"])
    
    for url in  url_list:
        response = requests.get(url)
        full_html = bs(response.content, "html.parser")
        Disease = full_html.find("h1").text
        result = u""

        try:    
            for tag in full_html.find("h2",text="Symptoms").next_siblings:
                if tag.name == "h2":
                    break
                else:
                    result += str(tag)
                        
            cropped_html = bs(result)
        
            row = ['']*18
            row[0] = Disease
            try:
                for ul in cropped_html.find_all('ul'):
                    for idx, li in enumerate(ul.findChildren('li')):
                        if idx == 17:
                            break
                        row[idx+1] = li.text
                    dataset.writerow(row)
            except:
                print(url)
        except:
            print(url)


https://www.mayoclinic.org/diseases-conditions/benign-adrenal-tumors/symptoms-causes/syc-20356190
https://www.mayoclinic.org/diseases-conditions/nickel-allergy/symptoms-causes/syc-20351529
https://www.mayoclinic.org/diseases-conditions/anal-fistula/symptoms-causes/syc-20352871
https://www.mayoclinic.org/diseases-conditions/airplane-ear/symptoms-causes/syc-20351701
https://www.mayoclinic.org/diseases-conditions/childhood-asthma/symptoms-causes/syc-20351507
https://www.mayoclinic.org/diseases-conditions/churg-strauss-syndrome/symptoms-causes/syc-20353760
https://www.mayoclinic.org/diseases-conditions/giant-cell-arteritis/symptoms-causes/syc-20372758
https://www.mayoclinic.org/diseases-conditions/antibiotic-associated-diarrhea/symptoms-causes/syc-20352231
https://www.mayoclinic.org/diseases-conditions/adrenoleukodystrophy/symptoms-causes/syc-20369157
https://www.mayoclinic.org/diseases-conditions/primary-progressive-aphasia/symptoms-causes/syc-20350499
https://www.mayoclinic.org/diseases-

In [44]:
#text = [[li.text for li in ul.find_all('li')] for ul in cropped_html.find_all('ul')]
#print(text)

TypeError: list indices must be integers or slices, not list

In [ ]:
#with open("csv/ebay_laptop_details.csv", mode="w", newline="", encoding='utf-8') as out_file:
#    ebay_info = csv.writer(out_file, delimiter=",", quotechar='"', quoting = csv.QUOTE_MINIMAL)
#    ebay_info.writerow(["title", "condition", "available_count", "sold_count", "price", "converted_price", "shipping_cost", "why_to_buy"])

    for url in  item_url_list:
        
        response = requests.get(url)
        print(response)            
        
        if response.status_code == 200:
            html_page = bs(response.content, "html.parser")
        
            symptom = html_page.find("h2",text="Symptoms")
            causes = html_page.find("h2",text="Causes")
        
            condition = html_page.find("div", {"id":"vi-itm-cond"})
            if condition != None:
                condition = condition.text
            
            available_count = html_page.find("span",{"id":"qtySubTxt"})
            if available_count != None:
                available_count = available_count.find("span").text
                av_regex = r"\n\t\t\t\t\t\t\t\t(.*) (.*?)$"
                available_count = re.findall(av_regex,available_count)[0][0]


#            ebay_info.writerow([title, condition, available_count, sold_count, price, price_in_usd, shipping_cost, w2b_content_dict])
        print("{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}".format(Disease, Symptom_1, Symptom_2, Symptom_3, Symptom_4, Symptom_5, Symptom_6, Symptom_7, Symptom_8, Symptom_9, Symptom_10, Symptom_11, Symptom_12, Symptom_13, Symptom_14, Symptom_15, Symptom_16, Symptom_17))

In [4]:
item_url_list = set()

def capture_product_links():
    all_links = driver.find_elements_by_tag_name('a');
    
    for link in all_links:
        href = link.get_attribute('href')
        if href.startswith("https://www.ebay.com/itm/"):
            item_url_list.add(href)
            
        if href.startswith("https://www.ebay.com/p/"):
            sub_driver = webdriver.Chrome(driver_exe_path)
            sub_driver.get(href)
            sleep(2)
            href = sub_driver.find_element_by_partial_link_text("See details").get_attribute('href')
            if href.startswith("https://www.ebay.com/itm/"):
                item_url_list.add(href)
            sub_driver.quit()
            

In [5]:
# route to electronics
stat_link = driver.find_element_by_link_text("Electronics")
stat_link.click()
sleep(2)

# route to computers
stat_link = driver.find_element_by_link_text("Computers, Tablets & Network Hardware")
stat_link.click()
sleep(2)

# route to laptops
stat_link = driver.find_element_by_link_text("Laptops & Netbooks")
stat_link.click()
sleep(2)

# route to PCs
stat_link = driver.find_element_by_link_text("PC Laptops & Netbooks")
stat_link.click()
sleep(2)

# filter to new PCs
# stat_link = driver.find_element_by_id("s0-29-13_2-0-1[1]-0-6-0-multiselect[]")
# stat_link.click()
# sleep(2)

print(driver.current_url)

https://www.ebay.com/b/PC-Laptops-Netbooks/177/bn_317584


In [6]:
def traverse_and_search_links():
    prev_list = -1
    current_list = 0

    #     use condition current_list != prev_list to go through pagination & capture all 
    while int(current_list) < 100:
        capture_product_links()
        prev_list = current_list
        next_button = driver.find_element_by_css_selector("a[rel='next']")
        next_button.click()
        sleep(2)
        current_list = driver.find_element_by_css_selector("li[class='ebayui-pagination__li ebayui-pagination__li--selected'] a").text

In [7]:
traverse_and_search_links()

print(len(item_url_list))
# print("\n".join(item_url_list))

1587


In [ ]:
with open("csv/ebay_laptop_details.csv", mode="w", newline="", encoding='utf-8') as out_file:
    ebay_info = csv.writer(out_file, delimiter=",", quotechar='"', quoting = csv.QUOTE_MINIMAL)
    ebay_info.writerow(["title", "condition", "available_count", "sold_count", "price", "converted_price", "shipping_cost", "why_to_buy"])

    for url in  item_url_list:
        
        response = requests.get(url)
        print(response)            
        
        if response.status_code == 200:
            html_page = bs(response.content, "html.parser")
        
            title = html_page.find("h1",{"id":"itemTitle"}).text
        
            condition = html_page.find("div", {"id":"vi-itm-cond"})
            if condition != None:
                condition = condition.text
            
            available_count = html_page.find("span",{"id":"qtySubTxt"})
            if available_count != None:
                available_count = available_count.find("span").text
                av_regex = r"\n\t\t\t\t\t\t\t\t(.*) (.*?)$"
                available_count = re.findall(av_regex,available_count)[0][0]
            
            sold_count = html_page.find("span",{"class":["vi-qtyS", "vi-qtyS-hot-red"]})
            if sold_count != None:
                sold_count= sold_count.find("a").text
                sc_regex = r"(.*?) sold?"
                sold_count = re.findall(sc_regex, sold_count)[0]
            
            price = html_page.find("span", {"id":"prcIsum"})
            if price != None:
                price = price.text
            
            price_in_usd = html_page.find("span", {"id":["convbinPrice", "convbidPrice"]})
            if price_in_usd != None:
                price_in_usd = price_in_usd.text
                pid_regex = r"US (.*?)\(including shipping\)?"
                price_in_usd = re.findall(pid_regex, price_in_usd)[0]
            
            shipping_cost = html_page.find("span",{"id":"fshippingCost"})
            if shipping_cost != None:
                shipping_cost = shipping_cost.find("span").text
            
            why_to_buy = html_page.find("div", {"id":"why2buy"})
            w2b_content_dict = {}
            if why_to_buy != None:
                why_to_buy_contents = why_to_buy.find_all("div",{"class":"w2b-cnt"})
            
                for content in why_to_buy_contents:
                    content_heading = content.find("span",{"class":"w2b-head"})
                    content_meaning = content.find("span",{"class":"w2b-subhead"})
                
                    if content_heading != None:
                        content_heading = content_heading.text
                    if content_meaning != None:
                        content_meaning = content_meaning.text
                    
                    w2b_content_dict[content_meaning] = content_heading

            ebay_info.writerow([title, condition, available_count, sold_count, price, price_in_usd, shipping_cost, w2b_content_dict])
#         print("{}, {}, {}, {}, {}, {}, {}".format(title, condition, available_count, sold_count, price, shipping_cost, watchers))